In [1]:
import weakref

# The class to be cached
class Spam:
    def __init__(self, name):
        print(f"Initializing Spam with name: {name}")
        self.name = name

# Global cache using WeakValueDictionary
_spam_cache = weakref.WeakValueDictionary()

def get_spam(name):
    if name not in _spam_cache:
        s = Spam(name)
        _spam_cache[name] = s
    else:
        s = _spam_cache[name]
    return s

# Example usage
a = get_spam("foo")
b = get_spam("bar")
c = get_spam("foo")

print(a is b)  # False
print(a is c)  # True


Initializing Spam with name: foo
Initializing Spam with name: bar
False
True


In [2]:
import weakref

# Manager class for caching
class CachedSpamManager:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()

    def get_spam(self, name):
        if name not in self._cache:
            s = Spam(name)
            self._cache[name] = s
        else:
            s = self._cache[name]
        return s

    def clear(self):
        self._cache.clear()

# The main class
class Spam:
    manager = CachedSpamManager()

    def __init__(self, name):
        print(f"Initializing Spam with name: {name}")
        self.name = name

def get_spam(name):
    return Spam.manager.get_spam(name)

# Example usage
a = get_spam("foo")
b = get_spam("bar")
c = get_spam("foo")

print(a is b)  # False
print(a is c)  # True


Initializing Spam with name: foo
Initializing Spam with name: bar
False
True


In [3]:
import weakref

# Manager class for caching
class CachedSpamManager:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()

    def get_spam(self, name):
        if name not in self._cache:
            s = Spam._new(name)
            self._cache[name] = s
        else:
            s = self._cache[name]
        return s

    def clear(self):
        self._cache.clear()

# The main class with restricted instantiation
class Spam:
    manager = CachedSpamManager()

    def __init__(self, *args, **kwargs):
        raise RuntimeError("Direct instantiation not allowed. Use get_spam().")

    @classmethod
    def _new(cls, name):
        instance = cls.__new__(cls)
        instance.name = name
        return instance

def get_spam(name):
    return Spam.manager.get_spam(name)

# Example usage
a = get_spam("foo")
b = get_spam("bar")
c = get_spam("foo")

print(a is b)  # False
print(a is c)  # True


False
True


In [4]:
import weakref

class CachedInstanceMeta(type):
    def __init__(cls, name, bases, dct):
        super().__init__(name, bases, dct)
        cls._cache = weakref.WeakValueDictionary()

    def __call__(cls, *args, **kwargs):
        key = (args, frozenset(kwargs.items()))
        if key not in cls._cache:
            instance = super().__call__(*args, **kwargs)
            cls._cache[key] = instance
        return cls._cache[key]

# The main class using the metaclass
class Spam(metaclass=CachedInstanceMeta):
    def __init__(self, name):
        print(f"Initializing Spam with name: {name}")
        self.name = name

# Example usage
a = Spam("foo")
b = Spam("bar")
c = Spam("foo")

print(a is b)  # False
print(a is c)  # True


Initializing Spam with name: foo
Initializing Spam with name: bar
False
True
